# Como trabalhar com stream

Ver [essa explicação](https://docs.databricks.com/spark/latest/structured-streaming/production.html) para mais detalhes.

Em resumo é recomendável utilizar um local para armazenar checkpoints assim você pode reiniciar a query de onde parou em caso de falhas.
Esse local de checkpoint preserva toda informação essencial que identifica unicamente uma query. _Cada query deve ter um local de checkpoint exclusivo_.

In [ ]:
streamingDataFrame.writeStream
  .format("parquet")
  .option("path", "dbfs://outputPath/")
  .option("checkpointLocation", "dbfs://checkpointPath")
  .start()

# Exemplo de stream



In [ ]:
# preciso criar um exemplo melhor para isso.
# esse caso ele avalia os arquivos que chegaram na entrada e processa automaticamente a cada execução.
# o stream não precisa ficar ligado o tempo inteiro.

INPUT_PATH = 'path/to/data'
OUTPUT_PATH = 'path/to/data/output'
CHECKPOINT_LOCATION_PATH = 'path/to/checkpoint'

schema_tags = """
tag string,
time string,
value string,
Year string,
Month string,
Date date
"""

dfStream = spark.readStream.format("cloudFiles") \
  .option("cloudFiles.format", "csv") \
  .option('sep', '|') \
  .schema(schema_tags) \
  .load(INPUT_PATH)

def foreachBatchWriteCleansed(df, id):
  # Validacao  se existem novos registros a serem processados
  if df.first() is None:
    return None

# Criacao da lista de novas datas a serem inseridas (reduzir o numero de arquivos lidos pelo merge)
  # https://kb.databricks.com/delta/delta-merge-into.html
  datas = [x.Date for x in df_final.select(col("Date").cast("string")).distinct().collect()]
  
  datas = ','.join([f'"{x}"' for x in datas]) # Ex: datas = '"2020-01-01", "2020-01-02"'

def create_tags_table_if_not_exist():
  sql(f"""CREATE TABLE IF NOT EXISTS out_table
  (
  id string,
  time timestamp,
  value string,
  Year integer,
  Month string,
  Date date
  )
  USING DELTA 
  PARTITIONED BY (Year, Month, Date)
  LOCATION '{OUTPUT_PATH}'"""
     )


  df.createOrReplaceTempView('temp_in')
  
  mergeQuery = f"""
  MERGE INTO out_table target
  USING temp_in new
  ON new.id = target.id
  AND new.time = target.time
  AND new.date = target.date
  AND target.date in ({datas})
  WHEN MATCHED
  THEN UPDATE SET *
  WHEN NOT MATCHED
  THEN INSERT *
  """
  df._jdf.sparkSession().sql(mergeQuery)

  return None

(
  dfStream.writeStream
  .trigger(once=True)
  .foreachBatch(foreachBatchWriteCleansed)
  .option("checkpointLocation", CHECKPOINT_LOCATION_PATH)
  .start()
  .awaitTermination()
)
